In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from matplotlib.ticker import FormatStrFormatter

In [2]:
#Definir valores a usar

#Definimos los valores para el ejercicio
codigo = "2191437"
X = sum ([int(i) for i in codigo])

#Definimos las caracteristicas de los cables
Longitud_Cable = 1.5 #m
Diametro_Cable = 1/4 #pulg

#Definimos las caracteristicas de los pasadores
Diametro_Pasadores = 1/2 #pulg

# Definimos las caracteristicas de la viga.
Longitud_Viga = X * 0.2 #m

#Definimos las propiedades del material de la viga
Esfuerzo_Ultimo = 400 #MPa
Esfuerzo_limite_elasticidad_tension = 250 #MPa
Esfuerzo_limite_elasticidad_cortante = 145 #MPa
Módulo_de_elasticidad = 200 #GPa

In [3]:
#Realizamos los calculos basicos necesarios (areas,angulos en radianes, factores de seguridad y conversiones de unidades)
Area_Pasadores = np.pi * ((Diametro_Pasadores/2) * 25.4 / 1000) ** 2 #mm2 
Area_Cables = np.pi * ((Diametro_Cable/2) * 25.4/ 1000) ** 2 #mm2
FS_Fu = float (f"1.{X}")
FS_Fy = float (f"1.{X+3}")
Angulo_CBD = 40 * (np.pi/180)
Angulo_DBE = 50 * (np.pi/180)

In [4]:
#Analizamos el equilibrio externo de la viga
Ax, Ay, BD, BC, BE, W = sp.symbols("Ax Ay BD BC BE W")

#Fy = 0
Fy = Ay - (W * Longitud_Viga) + BD + (BE * sp.cos(Angulo_DBE)) + (BC * sp.cos(Angulo_CBD))
sum_Fy = sp.Eq(Fy, 0)
sum_Fy

Eq(Ay + 0.766044443118978*BC + BD + 0.642787609686539*BE - 5.4*W, 0)

In [5]:
#Fx = 0
Fx = Ax + (BE * sp.sin(Angulo_DBE)) - (BC * sp.sin(Angulo_CBD))
sum_Fx = sp.Eq(Fx, 0)
sum_Fx

Eq(Ax - 0.642787609686539*BC + 0.766044443118978*BE, 0)

In [6]:
#Momento_A = 0
Ma = - W * (Longitud_Viga ** 2) / 2 + Longitud_Viga * BD + Longitud_Viga * BE * sp.cos(Angulo_DBE) + Longitud_Viga * BC * sp.cos(Angulo_CBD)
sum_Ma = sp.Eq(Ma, 0)
sum_Ma 

Eq(4.13663999284248*BC + 5.4*BD + 3.47105309230731*BE - 14.58*W, 0)

In [7]:
#Hacemos un analisis a las deformaciones de la viga

#Relacion entre BC y BD
BC_BD = sp.Eq(BC - BD * sp.cos(Angulo_CBD), 0)
Relacion_BC_BD = BD * sp.cos(Angulo_CBD)
BC_BD

Eq(BC - 0.766044443118978*BD, 0)

In [8]:
#Relacion entre BD y BE
BD_BE = sp.Eq(BE - BD * sp.cos(Angulo_DBE), 0)
Relacion_BD_BE = BD * sp.cos(Angulo_DBE)
BD_BE

Eq(-0.642787609686539*BD + BE, 0)

In [9]:
# Despejamos las ecuaciones en terminos de las reacciones o cargas

# BC y BE en terminos de BD
Momento = sp.Eq(Ma.subs([(BC, Relacion_BC_BD), (BE, Relacion_BD_BE)]), 0)
Momento

Eq(10.8*BD - 14.58*W, 0)

In [10]:
# BD en terminos de W
BD_en_W = sp.solve(Momento)[0][BD]
BD_en_W 

1.35*W

In [11]:
# Calculamos las cargas maximas para cada elemento

# W_max Cable_BD
Esfuerzo_BD = (BD_en_W / Area_Cables) - (Esfuerzo_Ultimo * (10 ** 6) / FS_Fu) / 1000
Eqv_BD_W = sp.Eq(Esfuerzo_BD, 0)
Eqv_BD_W
W_max_BD = sp.solve(Eqv_BD_W, W)
print("La carga máxima distribuida que puede soportar el cable BD es de " + str(W_max_BD[0]) + " kN/m")


La carga máxima distribuida que puede soportar el cable BD es de 7.38856050010933 kN/m


In [35]:
# W_max Cable_BC
Esfuerzo_BC = (Relacion_BC_BD.subs(BD, BD_en_W) / Area_Cables) - (Esfuerzo_Ultimo * (10 ** 6) / FS_Fu) / 1000
Eqv_BC_BD = sp.Eq(Esfuerzo_BC, 0)
Eqv_BC_BD
W_max_BC = sp.solve(Eqv_BC_BD, W)
print("La carga máxima distribuida que puede soportar el cable BD es de " + str(W_max_BC[0]) + " kN/m")

La carga máxima distribuida que puede soportar el cable BD es de 9.64508073451527 kN/m


In [36]:
# W_max Cable_BE
Esfuerzo_BE = (Relacion_BD_BE.subs(BD, BD_en_W) / Area_Cables) - (Esfuerzo_Ultimo * (10 ** 6) / FS_Fu) / 1000
Eqv_BE_BD = sp.Eq(Esfuerzo_BE, 0)
Eqv_BE_BD
W_max_BE = sp.solve(Eqv_BE_BD, W)
print("La carga máxima distribuida que puede soportar el cable BD es de " + str(W_max_BE[0]) + " kN/m")

La carga máxima distribuida que puede soportar el cable BD es de 11.4945596162198 kN/m


In [45]:
# PASADORES
# Pas_A
RAy = Fy.subs([(BC, Relacion_BC_BD), (BE, Relacion_BD_BE), (BD, BD_en_W)])
equivalencia_RAy = sp.solve(sp.Eq(RAy, 0))
Eqv_RAy = equivalencia_RAy[0]
Eqv_RAy

{Ay: 2.7*W}

In [46]:
RAx = Fx.subs([(BC, Relacion_BC_BD), (BE, Relacion_BD_BE), (BD, BD_en_W)])
Eqv_RAx = sp.solve(sp.Eq(RAx, 0))
Eqv_RAx = Eqv_RAx[0]
Eqv_RAx

{Ax: -1.49880108324396e-16*W}

In [55]:
# RA_en_W
Eqv_RAy_2 = 2.7*W ** 2
Eqv_RAx_2 = -1.49880108324396e-16*W ** 2
RA = sp.sqrt( Eqv_RAy_2 + Eqv_RAx_2 )
RA

1.6431676725155*sqrt(W**2)

In [57]:
# W_max Pas_A
Esfuerzo_RA = (RA / Area_Pasadores) - (Esfuerzo_limite_elasticidad_cortante * (10 ** 6) / FS_Fy) / 1000
Eqv_RA = sp.Eq(Esfuerzo_RA, 0)
Eqv_RA
W_1 = sp.sqrt(W**2)
W_max_RA = sp.solve(Eqv_RA, W_1)
print("La carga máxima distribuida que puede soportar el pasador A es de " + str(W_max_RA[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador A es de 8.59884441708339 kN/m


In [58]:
# Pas_B
RBy = Relacion_BC_BD.subs(BD, BD_en_W) * sp.cos(Angulo_CBD) + Relacion_BD_BE.subs(BD, BD_en_W) * sp.cos(Angulo_DBE) + BD_en_W
RBx = Relacion_BD_BE.subs(BD, BD_en_W) * sp.sin(Angulo_DBE) - Relacion_BC_BD.subs(BD, BD_en_W)* sp.sin(Angulo_CBD) 
RB = sp.sqrt(RBx**2 + RBy**2)
RB

2.7*sqrt(W**2)

In [61]:
# W_max Pas_B
Esfuerzo_RB = (RB / Area_Pasadores) - (Esfuerzo_limite_elasticidad_cortante * (10 ** 6)/ FS_Fy) / 1000
Eqv_RB = sp.Eq(Esfuerzo_RB, 0)
Eqv_RB
W_1 = sp.sqrt(W ** 2)
W_max_RB = sp.solve(Eqv_RB, W_1)
print("La carga máxima distribuida que puede soportar el pasador B es de " + str(W_max_RB[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador B es de 5.23309006190439 kN/m


In [62]:
# W_max Pas_BC
Esfuerzo_Pas_BC = (Relacion_BC_BD.subs(BD, BD_en_W) / Area_Pasadores) - (Esfuerzo_limite_elasticidad_cortante * (10 ** 6) / FS_Fy) / 1000
Eqv_Pas_BC = sp.Eq(Esfuerzo_Pas_BC, 0)
Eqv_Pas_BC
W_max_Pas_BC = sp.solve(Eqv_Pas_BC, W)
print("La carga máxima distribuida que puede soportar el pasador del cable BC es de " + str(W_max_Pas_BC[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador del cable BC es de 13.6626278250846 kN/m


In [63]:
# W_max Pas_BD
Esfuerzo_Pas_BD = (BD_en_W / Area_Pasadores) - (Esfuerzo_limite_elasticidad_cortante * (10 ** 6) / FS_Fy) / 1000
equivalencia_P_F_BD = sp.Eq(Esfuerzo_Pas_BD, 0)
equivalencia_P_F_BD
W_max_Pas_BD = sp.solve(equivalencia_P_F_BD, W)
print("La carga máxima distribuida que puede soportar el pasador del cable BD es de " + str(W_max_Pas_BD[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador del cable BD es de 10.4661801238088 kN/m


In [64]:
# W_max Pas_BE
Esfuerzo_Pas_BE = (Relacion_BD_BE.subs(BD, BD_en_W) / Area_Pasadores) - (Esfuerzo_limite_elasticidad_cortante * (10 ** 6) / FS_Fy) / 1000
Eqv_Pas_BE = sp.Eq(Esfuerzo_Pas_BE, 0)
Eqv_Pas_BE
W_max_Pas_BE = sp.solve(Eqv_Pas_BE, W)
print("La carga máxima distribuida que puede soportar el pasador del cable BE es de " + str(W_max_Pas_BE[0]) + " kN/m")

La carga máxima distribuida que puede soportar el pasador del cable BE es de 16.2824857948222 kN/m


In [66]:
# W_max general
W_max = min([W_max_BD, W_max_BC, W_max_BE, W_max_Pas_BC, W_max_Pas_BD, W_max_Pas_BE, W_max_RB, W_max_RA])
W_max_1 = W_max[0]
print("La carga máxima distribuida para que ningún elemento falle es de " + str(W_max) + " kN/m")

La carga máxima distribuida para que ningún elemento falle es de [5.23309006190439] kN/m


In [72]:
# 1.2 Reacciones, Fuerzas y Deformaciones

# Cable BD
W_BD = BD_en_W.subs(W, W_max_1)
Deformacion_BD = (W_BD * Longitud_Cable) / (Area_Cables * Módulo_de_elasticidad * 1000)
print("La fuerza del cable BD es de " + str(W_BD) + " kN")
print("La deformación del cable BD es de " + str(Deformacion_BD) + " mm")

La fuerza del cable BD es de 7.06467158357092 kN
La deformación del cable BD es de 1.67307692307693 mm


In [74]:
# Cable BC
W_BC = Relacion_BC_BD.subs(BD, BD_en_W)
W_BC = W_BC.subs(W, W_max_1)
Deformacion_BC = (W_BC * Longitud_Cable) / (Area_Cables * Módulo_de_elasticidad * 1000)
print("La fuerza del cable BC es de " + str(W_BC) + " kN")
print("La deformación del cable BC es de " + str(Deformacion_BC) + " mm")

La fuerza del cable BC es de 5.41185240905506 kN
La deformación del cable BC es de 1.28165127983368 mm


In [75]:
# Cable BE
W_BE = Relacion_BD_BE.subs(BD, BD_en_W)
W_BE = W_BE.subs(W, W_max_1)
Deformacion_BE = (W_BE * Longitud_Cable) / (Area_Cables * Módulo_de_elasticidad * 1000)
print("La fuerza del cable BC es de " + str(W_BE) + " kN")
print("La deformación del cable BE es de " + str(Deformacion_BE) + " mm")

La fuerza del cable BC es de 4.54108336042397 kN
La deformación del cable BE es de 1.07543311620633 mm


In [86]:
# Apoyo A
R_Ax = Fx.subs([(BC, W_BC), (BE, W_BE)])
R_Ax = sp.solve(sp.Eq(R_Ax,0))[0]
R_Ay = RA.subs([(Eqv_RAx, R_Ax), (W, W_max_1)])
print("La reaccion Ax es de " + str(R_Ax) + " kN")
print("La reaccion Ay es de " + str(R_Ay) + " kN")

La reaccion Ax es de -8.88178419700125e-16 kN
La reaccion Ay es de 8.59884441708342 kN
